<a href="https://colab.research.google.com/github/PreyPython123/Master-V24-Semiveiledet-Regresjon/blob/brixadjusted-Pradeep/bioco_brixadjusted_veiledet_regresjonsmetoder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


# Importering, lasting og formattering

Lasting av nødvendige bibliotek og pakker

In [ ]:
!pip install optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.1/380.1 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.4/233.4 kB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 8.5 MB/s eta 0:00:00


Importering av nødvendig bibliotek og pakker

In [ ]:
import pandas as pd
import numpy as np
import copy
import math
from statistics import mean, stdev
import time

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.pipeline import Pipeline

In [ ]:
# Modeller
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR

In [ ]:
# Hyperoptimalisering
import optuna
from optuna.visualization import plot_optimization_history

# Utelukker utskriving av logg for hyperoptimalisering
# optuna.logging.set_verbosity(optuna.logging.WARNING)

# Egendefinerte moduler

In [ ]:
!pwd
! cp /content/drive/MyDrive/MasterV24/GitHub/moduler_trening_validering_testing_evaluering.py /content/moduler_trening_validering_testing_evaluering.py

/content


In [ ]:
from moduler_trening_validering_testing_evaluering import trening_testsett_oppdeling_enzym
from moduler_trening_validering_testing_evaluering import trening_testsett_oppdeling_dag
from moduler_trening_validering_testing_evaluering import trening_validering_oppdeling_dag

from moduler_trening_validering_testing_evaluering import enzymfordeling
from moduler_trening_validering_testing_evaluering import n_splitt_enzym
from moduler_trening_validering_testing_evaluering import Resultat_evaluering
from moduler_trening_validering_testing_evaluering import res_eval_opt
from moduler_trening_validering_testing_evaluering import res_eval
from moduler_trening_validering_testing_evaluering import Resultat_dicts
from moduler_trening_validering_testing_evaluering import evaluering_beregning
from moduler_trening_validering_testing_evaluering import enzymtype_rawmaterial_prediksjon

In [ ]:
!pwd
! cp /content/drive/MyDrive/MasterV24/GitHub/moduler_optimalisering_evaluering_veiledet.py /content/moduler_optimalisering_evaluering_veiledet.py

/content


In [ ]:
from moduler_optimalisering_evaluering_veiledet import trening_rfr_enzym
from moduler_optimalisering_evaluering_veiledet import trening_knr_enzym
from moduler_optimalisering_evaluering_veiledet import trening_svr_enzym

from moduler_optimalisering_evaluering_veiledet import trening_rfr_dag
from moduler_optimalisering_evaluering_veiledet import trening_knr_dag
from moduler_optimalisering_evaluering_veiledet import trening_svr_dag

from moduler_optimalisering_evaluering_veiledet import veiledet_modell_evaluering_1
from moduler_optimalisering_evaluering_veiledet import veiledet_modell_evaluering_2

# Importering av relevant data

In [ ]:
# Velger første kolonne med dato og tid som index
data = pd.read_csv('/content/drive/MyDrive/MasterV24/Bioco_data/brixadjusted_data.csv',
                            header=0,
                            sep=',',
                            index_col=0)

# Design produksjon
data_design = pd.read_csv('/content/drive/MyDrive/MasterV24/Bioco_data/brixadjusted_data_design.csv',
                                   header=0,
                                   sep=',',
                                   index_col=0)

# Importerer rådata
rå_data = pd.read_csv('/content/drive/MyDrive/MasterV24/BiocoData.csv',
                      header=0,
                      sep=';',
                      index_col=0)

# Formatterer index til riktig format og datatype
data.index = pd.to_datetime(data.index,
                            format='%Y-%m-%d %H:%M:%S')

data_design.index = pd.to_datetime(data_design.index,
                                   format='%Y-%m-%d %H:%M:%S')

rå_data.index = pd.to_datetime(rå_data.index,
                               format='%d-%m-%Y %H:%M:%S.%f')

# Lagring av resultater

In [ ]:
# Oppretter instans for lagring av resultater
res = Resultat_dicts()

# Parametere til hyperoptimalisering

In [ ]:
# Antall runder til optimalisering
n_runder = 100

# Parametervalg for kryssvalidering
n_gjentagelser = 2

Tilfeldighetsfrø

In [ ]:
#Tilfeldighetsfrø
random_seed = 2024

# NIR målinger inkludert

## Hele datasett

## Alternativ 1: 20% Testsett og Hele resterende treningssett

In [ ]:
# Setter fast testandel og treningsandel
test_andel = 0.2
trenings_andel = 1.0


# Deler datasettet i trening og testsett
trening_markert, enzymtyper_markert, testsett, _, _, _, trening\
= trening_testsett_oppdeling_enzym(rå_data,
                                   data,
                                   test_andel=test_andel)

In [ ]:
n_splitt = n_splitt_enzym(rå_data,
                          trening_markert)

rskf = RepeatedStratifiedKFold(n_splits=n_splitt,
                                 n_repeats=n_gjentagelser,
                                 random_state=random_seed)

statistisk_res = Resultat_evaluering()
# Går igjennom kryssvalidering
for train, val in rskf.split(trening_markert,
                              enzymtyper_markert):
  # Markerte folds
  trenings_fold_markert = trening_markert.iloc[train, :]
  val_fold = trening_markert.iloc[val, :]

  # Markert respons
  y_trening_fold = trenings_fold_markert.iloc[:, -1]
  y_val_fold = val_fold.iloc[:, -1]


  y_trening_pred, y_trening_fold = enzymtype_rawmaterial_prediksjon(rå_data, trenings_fold_markert, trenings_fold_markert, 'BrixAdjusted', 'mean')
  y_val_pred, y_val_fold = enzymtype_rawmaterial_prediksjon(rå_data, trenings_fold_markert, val_fold, 'BrixAdjusted', 'mean')
  y_test_pred, y_test = enzymtype_rawmaterial_prediksjon(rå_data, trenings_fold_markert, testsett, 'BrixAdjusted', 'mean')


  statistisk_res.evaluering(y_trening_fold,
                            y_trening_pred,
                            y_val_fold,
                            y_val_pred,
                            y_test,
                            y_test_pred)

In [ ]:
statistisk_res.beskriv_statistikk()

RMSE (trening): Mean = 0.0120, Std = 0.0002
RMSE (val): Mean = 0.0121, Std = 0.0017
RMSE (test): Mean = 0.0125, Std = 0.0000
R2 (trening): Mean = 0.6190, Std = 0.0118
R2 (val): Mean = 0.5908, Std = 0.1157
R2 (test): Mean = 0.5339, Std = 0.0031
MAE (trening): Mean = 0.0081, Std = 0.0002
MAE (val): Mean = 0.0083, Std = 0.0011
MAE (test): Mean = 0.0082, Std = 0.0001
MAPE (trening): Mean = 0.0842, Std = 0.0023
MAPE (val): Mean = 0.0864, Std = 0.0193
MAPE (test): Mean = 0.0819, Std = 0.0004


In [ ]:
# Finner antall av enzymtype med minst antall
n_splitt = n_splitt_enzym(rå_data,
                          trening_markert)

# Modelloptimalisering, trening og evaluering av RFR, KNR og SVR
veiledet_modell_evaluering_1(dict_res_opt=res.dict_res_1_hel_opt,
                             dict_res=res.dict_res_1_hel,
                             treningssett_markert=trening_markert,
                             enzymtyper_treningssett_markert=enzymtyper_markert,
                             testsett=testsett,
                             n_splitt=n_splitt,
                             n_gjentagelser=n_gjentagelser,
                             n_runder=n_runder,
                             test_andel=test_andel,
                             trenings_andel=trenings_andel,
                             random_seed=random_seed)

# Lagring av resultater
res_1_hel = pd.DataFrame(res.dict_res_1_hel)
res_1_hel_opt = pd.DataFrame(res.dict_res_1_hel_opt)

res_1_hel.to_csv('/content/drive/MyDrive/MasterV24/Bioco_data/Resultater/brixadjusted/brixadjusted_resultater_1_hel.csv')
res_1_hel_opt.to_csv('/content/drive/MyDrive/MasterV24/Bioco_data/Resultater/brixadjusted/brixadjusted_resultater_1_hel_opt.csv')

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
[I 2024-06-06 09:21:39,370] A new study created in memory with name: no-name-07eb3e9a-6089-479a-818c-d9cbeffa85cc
[I 2024-06-06 09:21:39,816] Trial 1 finished with value: 0.009963357487746257 and parameters: {'n_neighbors': 5, 'weights': 'distance', 'p': 10}. Best is trial 1 with value: 0.009963357487746257.
[I 2024-06-06 09:21:39,820] Trial 0 finished with value: 0.010378335588644151 and parameters: {'n_neighbors': 9, 'weights': 'uniform', 'p': 7}. Best is trial 1 with value: 0.009963357487746257.
[I 2024-06-06 09:21:40,268] Trial 2 finished with value: 0.009817118685866122 and parameters: {'n_neighbors': 5, 'weights': 'uniform', 'p': 2

Velger beste modell

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


## Alternativ 2: 20% Testsett og resteremde hele treningssett

In [ ]:
# Definerer trenings- og testandel
test_andel = 0.2
trenings_andel = 1.0

# Deler datasettet i trening og testsett
trening_markert, testsett, _, trening\
= trening_testsett_oppdeling_dag(rå_data,
                                 data,
                                 test_andel=test_andel,
                                 test_ekskluder_kalkun_kylling=True)

In [ ]:
n_splitt = n_splitt_enzym(rå_data,
                          trening_markert)


statistisk_res = Resultat_evaluering()
# Går igjennom kryssvalidering
trenings_sett, val_sett\
= trening_validering_oppdeling_dag(rå_data,
                                    trening_markert,
                                    n_splitt=n_splitt,
                                    test_ekskluder_kalkun_kylling=True)

for i, j in zip(trenings_sett, val_sett):
    train = trenings_sett[i]
    val = val_sett[j]

    # Markerte folds
    trenings_fold_markert = trening_markert.loc[train, :]
    val_fold = trening_markert.loc[val, :]

    # Markert respons
    y_trening_fold = trenings_fold_markert.iloc[:, -1]
    y_val_fold = val_fold.iloc[:, -1]


    y_trening_pred, y_trening_fold = enzymtype_rawmaterial_prediksjon(rå_data, trenings_fold_markert, trenings_fold_markert, 'BrixAdjusted', 'mean')
    y_val_pred, y_val_fold = enzymtype_rawmaterial_prediksjon(rå_data, trenings_fold_markert, val_fold, 'BrixAdjusted', 'mean')
    y_test_pred, y_test = enzymtype_rawmaterial_prediksjon(rå_data, trenings_fold_markert, testsett, 'BrixAdjusted', 'mean')


    statistisk_res.evaluering(y_trening_fold,
                              y_trening_pred,
                              y_val_fold,
                              y_val_pred,
                              y_test,
                              y_test_pred)

In [ ]:
statistisk_res.beskriv_statistikk()

RMSE (trening): Mean = 0.0123, Std = 0.0003
RMSE (val): Mean = 0.0139, Std = 0.0031
RMSE (test): Mean = 0.0107, Std = 0.0001
R2 (trening): Mean = 0.6012, Std = 0.0168
R2 (val): Mean = 0.4373, Std = 0.2396
R2 (test): Mean = 0.6516, Std = 0.0088
MAE (trening): Mean = 0.0081, Std = 0.0002
MAE (val): Mean = 0.0102, Std = 0.0023
MAE (test): Mean = 0.0080, Std = 0.0001
MAPE (trening): Mean = 0.0852, Std = 0.0030
MAPE (val): Mean = 0.1069, Std = 0.0326
MAPE (test): Mean = 0.0740, Std = 0.0011


In [ ]:
# Finner antall av enzymtype med minst antall
n_splitt = n_splitt_enzym(rå_data,
                          trening_markert)

# Modelloptimalisering, trening og evaluering av RFR, KNR og SVR
veiledet_modell_evaluering_2(dict_res_opt=res.dict_res_2_hel_opt,
                             dict_res=res.dict_res_2_hel,
                             rå_data=rå_data,
                             treningssett_markert=trening_markert,
                             testsett=testsett,
                             n_splitt=n_splitt,
                             n_runder=n_runder,
                             test_andel=test_andel,
                             trenings_andel=trenings_andel,
                             random_seed=random_seed)

res_2_hel = pd.DataFrame(res.dict_res_2_hel)
res_2_hel_opt = pd.DataFrame(res.dict_res_2_hel_opt)

res_2_hel.to_csv('/content/drive/MyDrive/MasterV24/Bioco_data/Resultater/brixadjusted/brixadjusted_resultater_2_hel.csv')
res_2_hel_opt.to_csv('/content/drive/MyDrive/MasterV24/Bioco_data/Resultater/brixadjusted/brixadjusted_resultater_2_hel_opt.csv')

[I 2024-05-01 18:13:25,168] A new study created in memory with name: no-name-bf77a6f4-d3c6-4b48-8bca-30d51475f53a
[I 2024-05-01 18:13:37,694] Trial 0 finished with value: 0.011278197166668403 and parameters: {'n_neighbors': 11, 'weights': 'uniform', 'p': 1}. Best is trial 0 with value: 0.011278197166668403.
[I 2024-05-01 18:13:38,206] Trial 1 finished with value: 0.011201710701854073 and parameters: {'n_neighbors': 11, 'weights': 'distance', 'p': 5}. Best is trial 1 with value: 0.011201710701854073.
[I 2024-05-01 18:13:50,345] Trial 2 finished with value: 0.011558226558571054 and parameters: {'n_neighbors': 12, 'weights': 'uniform', 'p': 2}. Best is trial 1 with value: 0.011201710701854073.
[I 2024-05-01 18:13:51,269] Trial 3 finished with value: 0.01086487757004119 and parameters: {'n_neighbors': 4, 'weights': 'uniform', 'p': 9}. Best is trial 3 with value: 0.01086487757004119.
[I 2024-05-01 18:14:03,833] Trial 4 finished with value: 0.011674677099674142 and parameters: {'n_neighbors'

Velger beste modell

## Designproduksjon

## Alternativ 1: 20% Testsett og Hele resterende treningssett

In [ ]:
# Setter fast testandel og treningsandel
test_andel = 0.2
trenings_andel = 1.0

# Deler datasettet i trening og testsett
trening_markert, enzymtyper_markert, testsett, _, _, _, trening\
= trening_testsett_oppdeling_enzym(rå_data,
                                   data_design,
                                   test_andel=test_andel)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
# Finner antall av enzymtype med minst antall
n_splitt = n_splitt_enzym(rå_data,
                          trening_markert)

# Modelloptimalisering, trening og evaluering av RFR, KNR og SVR
veiledet_modell_evaluering_1(dict_res_opt=res.dict_res_1_design_opt,
                             dict_res=res.dict_res_1_design,
                             treningssett_markert=trening_markert,
                             enzymtyper_treningssett_markert=enzymtyper_markert,
                             testsett=testsett,
                             n_splitt=n_splitt,
                             n_gjentagelser=n_gjentagelser,
                             n_runder=n_runder,
                             test_andel=test_andel,
                             trenings_andel=trenings_andel,
                             random_seed=random_seed)

res_1_design = pd.DataFrame(res.dict_res_1_design)
res_1_design_opt = pd.DataFrame(res.dict_res_1_design_opt)

res_1_design.to_csv('/content/drive/MyDrive/MasterV24/Bioco_data/Resultater/brixadjusted/brixadjusted_resultater_1_design.csv')
res_1_design_opt.to_csv('/content/drive/MyDrive/MasterV24/Bioco_data/Resultater/brixadjusted/brixadjusted_resultater_1_design_opt.csv')

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
[I 2024-06-06 10:00:57,802] A new study created in memory with name: no-name-39c3b681-5bd4-49e4-8802-a7e57848d63f
[I 2024-06-06 10:00:58,218] Trial 0 finished with value: 0.005799999368069819 and parameters: {'n_neighbors': 7, 'weights': 'uniform', 'p': 6}. Best is trial 0 with value: 0.005799999368069819.
[I 2024-06-06 10:00:58,233] Trial 1 finished with value: 0.006100150264422189 and parameters: {'n_neighbors': 4, 'weights': 'distance', 'p': 6}. Best is trial 0 with value: 0.005799999368069819.
[I 2024-06-06 10:00:58,743] Trial 2 finished with value: 0.005774962750942384 and parameters: {'n_neighbors': 6, 'weights': 'distance', 'p': 3

Velger beste modell

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


## Alternativ 2: 20% Testsett og resteremde hele treningssett

In [ ]:
# Definerer trenings- og testandel
test_andel = 0.20
trenings_andel = 1.0


# Deler datasettet i trening og testsett
trening_markert, testsett, _, trening\
= trening_testsett_oppdeling_dag(rå_data,
                                 data,
                                 test_andel=test_andel,
                                 test_ekskluder_kalkun_kylling=True)

In [ ]:
# Finner antall av enzymtype med minst antall
n_splitt = n_splitt_enzym(rå_data,
                          trening_markert)

# Modelloptimalisering, trening og evaluering av RFR, KNR og SVR
veiledet_modell_evaluering_2(dict_res_opt=res.dict_res_2_design_opt,
                             dict_res=res.dict_res_2_design,
                             rå_data=rå_data,
                             treningssett_markert=trening_markert,
                             testsett=testsett,
                             n_splitt=n_splitt,
                             n_runder=n_runder,
                             test_andel=test_andel,
                             trenings_andel=trenings_andel,
                             random_seed=random_seed)

# Lagring av resultater
res_2_design = pd.DataFrame(res.dict_res_2_design)
res_2_design_opt = pd.DataFrame(res.dict_res_2_design_opt)

res_2_design.to_csv('/content/drive/MyDrive/MasterV24/Bioco_data/Resultater/brixadjusted/brixadjusted_resultater_2_design.csv')
res_2_design_opt.to_csv('/content/drive/MyDrive/MasterV24/Bioco_data/Resultater/brixadjusted/brixadjusted_resultater_2_design_opt.csv')

[I 2024-05-01 21:58:22,296] A new study created in memory with name: no-name-dfa4a02b-60fc-45f5-bd02-99069fbdd351
[I 2024-05-01 21:58:29,744] Trial 0 finished with value: 0.010651151809784012 and parameters: {'n_neighbors': 17, 'weights': 'distance', 'p': 1}. Best is trial 0 with value: 0.010651151809784012.
[I 2024-05-01 21:58:29,775] Trial 1 finished with value: 0.010325955290234258 and parameters: {'n_neighbors': 11, 'weights': 'distance', 'p': 2}. Best is trial 1 with value: 0.010325955290234258.
[I 2024-05-01 21:58:37,418] Trial 2 finished with value: 0.010288704428026263 and parameters: {'n_neighbors': 10, 'weights': 'distance', 'p': 2}. Best is trial 2 with value: 0.010288704428026263.
[I 2024-05-01 21:58:37,539] Trial 3 finished with value: 0.010325362631011237 and parameters: {'n_neighbors': 10, 'weights': 'distance', 'p': 5}. Best is trial 2 with value: 0.010288704428026263.
[I 2024-05-01 21:58:44,637] Trial 5 finished with value: 0.01228764674690111 and parameters: {'n_neigh

Velger beste modell

# NIR målinger ekskludert

In [ ]:
# Ekskluderer NIR målinger fra datasettene
NIR_kolonner = ['NIRfat', 'NIRwater', 'NIRash']
data_uten_NIR = data.drop(columns=NIR_kolonner)
data_design_uten_NIR = data_design.drop(columns=NIR_kolonner)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
# Setter fast testandel og treningsandel
test_andel = 0.2
trenings_andel = 1.0

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


## Hele datasett

## Alternativ 1: 20% Testsett og Hele resterende treningssett

In [ ]:
# Deler datasettet i trening og testsett
trening_markert, enzymtyper_markert, testsett, _, _, _, trening\
= trening_testsett_oppdeling_enzym(rå_data,
                                   data_uten_NIR,
                                   test_andel=test_andel)

# Finner antall av enzymtype med minst antall
n_splitt = n_splitt_enzym(rå_data,
                          trening_markert)

# Modelloptimalisering, trening og evaluering av RFR, KNR og SVR
veiledet_modell_evaluering_1(dict_res_opt=res.dict_res_1_hel_opt_NIR,
                             dict_res=res.dict_res_1_hel_NIR,
                             treningssett_markert=trening_markert,
                             enzymtyper_treningssett_markert=enzymtyper_markert,
                             testsett=testsett,
                             n_splitt=n_splitt,
                             n_gjentagelser=n_gjentagelser,
                             n_runder=n_runder,
                             test_andel=test_andel,
                             trenings_andel=trenings_andel,
                             random_seed=random_seed)

# Lagring av resultater
res_1_hel_NIR = pd.DataFrame(res.dict_res_1_hel_NIR)
res_1_hel_opt_NIR = pd.DataFrame(res.dict_res_1_hel_opt_NIR)

res_1_hel_NIR.to_csv('/content/drive/MyDrive/MasterV24/Bioco_data/Resultater/brixadjusted/brixadjusted_resultater_1_hel_NIR.csv')
res_1_hel_opt_NIR.to_csv('/content/drive/MyDrive/MasterV24/Bioco_data/Resultater/brixadjusted/brixadjusted_resultater_1_hel_opt_NIR.csv')

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
[I 2024-06-06 11:05:16,082] A new study created in memory with name: no-name-78533360-6f9d-4f48-9757-d856c3059418
[I 2024-06-06 11:05:16,464] Trial 0 finished with value: 0.010811781389222863 and parameters: {'n_neighbors': 3, 'weights': 'uniform', 'p': 2}. Best is trial 0 with value: 0.010811781389222863.
[I 2024-06-06 11:05:16,494] Trial 1 finished with value: 0.013350882232439444 and parameters: {'n_neighbors': 1, 'weights': 'distance', 'p': 9}. Best is trial 0 with value: 0.010811781389222863.
[I 2024-06-06 11:05:16,883] Trial 3 finished with value: 0.01040207013941753 and parameters: {'n_neighbors': 10, 'weights': 'distance', 'p': 1

Velger beste modell

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


## Alternativ 2: 20% Testsett og resteremde hele treningssett

In [ ]:
# Deler datasettet i trening og testsett
trening_markert, testsett, _, trening\
= trening_testsett_oppdeling_dag(rå_data,
                                 data_uten_NIR,
                                 test_andel=test_andel,
                                 test_ekskluder_kalkun_kylling=True)

# Finner antall av enzymtype med minst antall
n_splitt = n_splitt_enzym(rå_data,
                          trening_markert)

# Modelloptimalisering, trening og evaluering av RFR, KNR og SVR
veiledet_modell_evaluering_2(dict_res_opt=res.dict_res_2_hel_opt_NIR,
                             dict_res=res.dict_res_2_hel_NIR,
                             rå_data=rå_data,
                             treningssett_markert=trening_markert,
                             testsett=testsett,
                             n_splitt=n_splitt,
                             n_runder=n_runder,
                             test_andel=test_andel,
                             trenings_andel=trenings_andel,
                             random_seed=random_seed)

# Lagring av resulater
res_2_hel_NIR = pd.DataFrame(res.dict_res_2_hel_NIR)
res_2_hel_opt_NIR = pd.DataFrame(res.dict_res_2_hel_opt_NIR)

res_2_hel_NIR.to_csv('/content/drive/MyDrive/MasterV24/Bioco_data/Resultater/brixadjusted/brixadjusted_resultater_2_hel_NIR.csv')
res_2_hel_opt_NIR.to_csv('/content/drive/MyDrive/MasterV24/Bioco_data/Resultater/brixadjusted/brixadjusted_resultater_2_hel_opt_NIR.csv')

[I 2024-06-05 18:53:53,798] A new study created in memory with name: no-name-684b15ce-55b1-4c47-8c74-c5693f0931b6
[I 2024-06-05 18:54:11,268] Trial 1 finished with value: 0.011940794016951396 and parameters: {'n_neighbors': 7, 'weights': 'uniform', 'p': 2}. Best is trial 1 with value: 0.011940794016951396.
[I 2024-06-05 18:54:11,334] Trial 0 finished with value: 0.012210202114376673 and parameters: {'n_neighbors': 8, 'weights': 'uniform', 'p': 4}. Best is trial 1 with value: 0.011940794016951396.
[I 2024-06-05 18:54:29,463] Trial 3 finished with value: 0.011931649117070559 and parameters: {'n_neighbors': 3, 'weights': 'uniform', 'p': 3}. Best is trial 3 with value: 0.011931649117070559.
[I 2024-06-05 18:54:29,484] Trial 2 finished with value: 0.011948150943458935 and parameters: {'n_neighbors': 6, 'weights': 'uniform', 'p': 2}. Best is trial 3 with value: 0.011931649117070559.
[I 2024-06-05 18:54:47,453] Trial 5 finished with value: 0.01160938183729294 and parameters: {'n_neighbors': 4

Velger beste modell

## Designsett

## Alternativ 1: 20% Testsett og Hele resterende treningssett

In [ ]:
# Deler datasettet i trening og testsett
trening_markert, enzymtyper_markert, testsett, _, _, _, trening\
= trening_testsett_oppdeling_enzym(rå_data,
                                   data_design_uten_NIR,
                                   test_andel=test_andel)
# Finner antall av enzymtype med minst antall
n_splitt = n_splitt_enzym(rå_data,
                          trening_markert)

# Modelloptimalisering, trening og evaluering av RFR, KNR og SVR
veiledet_modell_evaluering_1(dict_res_opt=res.dict_res_1_design_opt_NIR,
                             dict_res=res.dict_res_1_design_NIR,
                             treningssett_markert=trening_markert,
                             enzymtyper_treningssett_markert=enzymtyper_markert,
                             testsett=testsett,
                             n_splitt=n_splitt,
                             n_gjentagelser=n_gjentagelser,
                             n_runder=n_runder,
                             test_andel=test_andel,
                             trenings_andel=trenings_andel,
                             random_seed=random_seed)

# Lagring av resultater
res_1_design_NIR = pd.DataFrame(res.dict_res_1_design_NIR)
res_1_design_opt_NIR = pd.DataFrame(res.dict_res_1_design_opt_NIR)

res_1_design_NIR.to_csv('/content/drive/MyDrive/MasterV24/Bioco_data/Resultater/brixadjusted/brixadjusted_resultater_1_design_NIR.csv')
res_1_design_opt_NIR.to_csv('/content/drive/MyDrive/MasterV24/Bioco_data/Resultater/brixadjusted/brixadjusted_resultater_1_design_opt_NIR.csv')

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
[I 2024-06-06 11:21:04,053] A new study created in memory with name: no-name-24f26d6e-f266-485b-b1be-c03d0ab6c91e
[I 2024-06-06 11:21:04,401] Trial 1 finished with value: 0.006309579673623226 and parameters: {'n_neighbors': 4, 'weights': 'distance', 'p': 2}. Best is trial 1 with value: 0.006309579673623226.
[I 2024-06-06 11:21:04,462] Trial 0 finished with value: 0.006284795811605605 and parameters: {'n_neighbors': 3, 'weights': 'uniform', 'p': 1}. Best is trial 0 with value: 0.006284795811605605.
[I 2024-06-06 11:21:04,763] Trial 2 finished with value: 0.0062944478461476 and parameters: {'n_neighbors': 3, 'weights': 'distance', 'p': 1}.

Velger beste modell

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


## Alternativ 2: 20% Testsett og resteremde hele treningssett

In [ ]:
# Deler datasettet i trening og testsett
trening_markert, testsett, _, trening\
= trening_testsett_oppdeling_dag(rå_data,
                                 data,
                                 test_andel=test_andel,
                                 test_ekskluder_kalkun_kylling=True)

# Finner antall av enzymtype med minst antall
n_splitt = n_splitt_enzym(rå_data,
                          trening_markert)

# Modelloptimalisering, trening og evaluering av RFR, KNR og SVR
veiledet_modell_evaluering_2(dict_res_opt=res.dict_res_2_design_opt_NIR,
                             dict_res=res.dict_res_2_design_NIR,
                             rå_data=rå_data,
                             treningssett_markert=trening_markert,
                             testsett=testsett,
                             n_splitt=n_splitt,
                             n_runder=n_runder,
                             test_andel=test_andel,
                             trenings_andel=trenings_andel,
                             random_seed=random_seed)

# Lagring av resultater
res_2_design_NIR = pd.DataFrame(res.dict_res_2_design_NIR)
res_2_design_opt_NIR = pd.DataFrame(res.dict_res_2_design_opt_NIR)

res_2_design_NIR.to_csv('/content/drive/MyDrive/MasterV24/Bioco_data/Resultater/brixadjusted/brixadjusted_resultater_2_design_NIR.csv')
res_2_design_opt_NIR.to_csv('/content/drive/MyDrive/MasterV24/Bioco_data/Resultater/brixadjusted/brixadjusted_resultater_2_design_opt_NIR.csv')

Velger beste modell